# Cineman Plotly Map

In [8]:
import pandas as pd
from datetime import date, time
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

In [59]:
path=f"../data/raw/{date.today()}_showtimes.csv"
cineman_df = pd.read_csv(path, index_col=0)
cineman_df.head()

,movie,genre,age_limit,language,showtime,date,cinema,place,cinema_place,latitude,longitude
0,Bigger Than Us,Documentary,10Y.,E/gf,11:00,2021-10-23,Arthouse Piccadilly,Zürich,Arthouse Piccadilly Zürich,47.365993,8.548905
1,Tiger und Büffel,Documentary,12Y.,O/g,11:50,2021-10-23,Houdini,Zürich,Houdini Zürich,47.374607,8.520307
2,"Quo Vadis, Aida?","Drama, War",14/12Y.,O/gf,12:00,2021-10-23,Houdini,Zürich,Houdini Zürich,47.374607,8.520307
3,The Story of My Wife,"Drama, Romance",12Y.,E/g,12:00,2021-10-23,Houdini,Zürich,Houdini Zürich,47.374607,8.520307
4,Wild,Documentary,12Y.,O/g,12:10,2021-10-23,Houdini,Zürich,Houdini Zürich,47.374607,8.520307


In [47]:
cineman_df.dtypes

movie                   object
genre                   object
age_limit               object
language                object
showtime                object
date                    object
cinema                  object
place                   object
cinema_place            object
latitude               float64
longitude              float64
dt_showtime     datetime64[ns]
dtype: object

In [53]:
hours = ["All", "9-12am", "12-3pm", "3-6pm", "6-9pm", "9-12pm"]

In [61]:
selected_hour = "9-12am"

if selected_hour == "All":
    cineman_df = cineman_df
elif selected_hour == "9-12am":
    cineman_df = cineman_df[cineman_df["dt_showtime"].between('09:00:00', '12:00:00')]
elif selected_hour == "12-3pm":
    cineman_df = cineman_df[cineman_df["dt_showtime"].between('12:00:00', '15:00:00')]
elif selected_hour == "3-6pm":
    cineman_df = cineman_df[cineman_df["dt_showtime"].between('15:00:00', '18:00:00')]    
elif selected_hour == "6-9pm":
    cineman_df = cineman_df[cineman_df["dt_showtime"].between('18:00:00', '21:00:00')]  
else:
    cineman_df = cineman_df[cineman_df["dt_showtime"].between('21:00:00', '23:59:00')]    

cineman_df

,movie,genre,age_limit,language,showtime,date,cinema,place,cinema_place,latitude,longitude,dt_showtime
66,Feuerpferde,NaN,unknown,O/gf,15:00,2021-10-23,Filmpodium,Zürich,Filmpodium Zürich,47.371574,8.536825,2021-10-23 15:00:00
67,Nachbarn,Drama,12Y.,O/gf,15:00,2021-10-23,Arthouse Movie,Zürich,Arthouse Movie Zürich,47.370704,8.543701,2021-10-23 15:00:00
68,Cry Macho,"Drama, Western",12Y.,E/gf,15:30,2021-10-23,Kosmos,Zürich,Kosmos Zürich,47.379981,8.529081,2021-10-23 15:30:00
69,Die Schule der magischen Tiere,"Adventure, Children",3Y.,G,15:30,2021-10-23,Kosmos,Zürich,Kosmos Zürich,47.379981,8.529081,2021-10-23 15:30:00
70,Die Schule der magischen Tiere,"Adventure, Children",3Y.,G,15:30,2021-10-23,Stüssihof,Zürich,Stüssihof Zürich,47.372324,8.543770,2021-10-23 15:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...
137,Dolci inganni,NaN,unknown,I/f,18:00,2021-10-23,Filmpodium,Zürich,Filmpodium Zürich,47.371574,8.536825,2021-10-23 18:00:00
138,Je suis Karl,Drama,12Y.,G,18:00,2021-10-23,Kosmos,Zürich,Kosmos Zürich,47.379981,8.529081,2021-10-23 18:00:00
139,Shang-Chi and the Legend of the Ten Rings,"Action, Adventure, Fantasy",12Y.,G,18:00,2021-10-23,Arena Cinemas,Zürich,Arena Cinemas Zürich,47.358460,8.522921,2021-10-23 18:00:00
140,The French Dispatch,"Comedy, Drama, Romance",14/12Y.,E/gf,18:00,2021-10-23,Arthouse Le Paris,Zürich,Arthouse Le Paris Zürich,47.366435,8.547469,2021-10-23 18:00:00


Adjusting the style in mapbox:
- "white-bg" yields an empty white canvas which results in no external HTTP requests
- "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor" yield maps composed of raster tiles from various public tile servers which do not require signups or access tokens
- "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets" yield maps composed of vector tiles from the Mapbox service, and do require a Mapbox Access Token or an on-premise Mapbox installation.
- A Mapbox service style URL, which requires a Mapbox Access Token or an on-premise Mapbox installation.
- A Mapbox Style object as defined at https://docs.mapbox.com/mapbox-gl-js/style-spec/


In [4]:
def create_plotly_map(df, access_token):
    fig = go.Figure(go.Scattermapbox(
        lat=df["latitude"],
        lon=df["longitude"],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10, color="crimson",
            ),
        
        text=df["cinema"],
        hoverinfo="text",

    ))

    fig.update_layout(
        hovermode='closest',
        width=800,
        height=800,
        margin={"r":0,"t":0,"l":0,"b":0},
        mapbox=dict(
            accesstoken=access_token,
            center=go.layout.mapbox.Center(
                lat=47.374,
                lon=8.535
            ),
            zoom=13,
            style="streets"
        )
    )

    fig.show()

In [5]:
mapbox_access_token = open("../.mapbox_token").read()
create_plotly_map(cineman_df, mapbox_access_token)

In [63]:
movie_desc = pd.read_csv(f"../data/raw/{date.today()}_zurich_movie_overviews.csv")

In [86]:
selected_movie = "Wild"

overview = movie_desc[movie_desc["original_title"] == movie]["overview"].values[0]
print(overview)

A woman with a tragic past decides to start her new life by hiking for one thousand miles on the Pacific Crest Trail.
